
https://www.kaggle.com/talfik2/recommendation-systems/notebook

# 1. Mount google drive

In [176]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [177]:
import os

os.chdir('/content/gdrive/My Drive/ISY5001ReasoningPM/colab/spotify_our_data/')
print("Current working directory:")
os.getcwd()

Current working directory:


'/content/gdrive/My Drive/ISY5001ReasoningPM/colab/spotify_our_data'

In [178]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import tensorflow as tf
import pickle

# 2. Data preparation

In [179]:
spoti = pd.read_csv("./dataset/spotifydata1026utf8.csv", encoding='utf-8', quotechar='"')
spoti.head(3)

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,song_name,artist,genre
0,0,0.741,0.739,11.0,-4.627,0.0,0.0386,0.005410,0.026900,0.0496,0.425,126.008,audio_features,0ikuoejgnBNitqFArW6fFk,spotify:track:0ikuoejgnBNitqFArW6fFk,https://api.spotify.com/v1/tracks/0ikuoejgnBNi...,https://api.spotify.com/v1/audio-analysis/0iku...,262857.0,4.0,Dark River - Festival Version,Sebastian Ingrosso,absolute music
1,1,0.649,0.973,6.0,-4.391,1.0,0.0882,0.000639,0.662000,0.1330,0.221,127.951,audio_features,4nrK0G6pOEuhMxn3hAaava,spotify:track:4nrK0G6pOEuhMxn3hAaava,https://api.spotify.com/v1/tracks/4nrK0G6pOEuh...,https://api.spotify.com/v1/audio-analysis/4nrK...,279375.0,4.0,Derp - Original Mix,Bassjackers,absolute music
2,2,0.791,0.862,9.0,-3.240,0.0,0.1100,0.037000,0.000022,0.0814,0.592,123.994,audio_features,7ef4DlsgrMEH11cDZd32M6,spotify:track:7ef4DlsgrMEH11cDZd32M6,https://api.spotify.com/v1/tracks/7ef4DlsgrMEH...,https://api.spotify.com/v1/audio-analysis/7ef4...,214847.0,4.0,One Kiss (with Dua Lipa),Calvin Harris,absolute music


In [180]:
spoti.shape

(10097, 22)

## Creat song name data

In [181]:
# screat song name
song_name = spoti["song_name"]

print(song_name.shape)
print(song_name.isnull().values.any())

(10097,)
True


In [182]:
song_name = song_name.values.reshape(-1,1)

song_name.shape

(10097, 1)

## Replace NA values

In [183]:
from sklearn.impute import SimpleImputer
imr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imr = imr.fit(song_name)
imputed_data = imr.transform(song_name)
song_name = pd.DataFrame(imputed_data)
song_name = song_name.rename(columns={0:"Song-Names"})
song_name.head(10)
song_name.values.tolist()[0]

['Dark River - Festival Version']

In [184]:
PIK = "name_pickle.dat"

with open(PIK, "wb") as f:
    pickle.dump(song_name, f)
with open(PIK, "rb") as f:
    reload_song_name = pickle.load(f)

reload_song_name.head(10)

,Song-Names
0,Dark River - Festival Version
1,Derp - Original Mix
2,One Kiss (with Dua Lipa)
3,Kill This Love
4,Physical
5,Turn Me On (feat. Vula)
6,Lightning Strikes
7,Invincible - Original Mix
8,Bella ciao - HUGEL Remix
9,SHIVA (Sunburn 2017 Anthem) [feat. The Golden ...


## Creat song name artists genre data

In [185]:
song_artist_map = spoti[["song_name", "genre","artist"]]
print(song_artist_map.dtypes)
print(song_artist_map.head())
# print(song_artist_map.index.values.tolist()) 0开始


song_name    object
genre        object
artist       object
dtype: object
                       song_name           genre              artist
0  Dark River - Festival Version  absolute music  Sebastian Ingrosso
1            Derp - Original Mix  absolute music         Bassjackers
2       One Kiss (with Dua Lipa)  absolute music       Calvin Harris
3                 Kill This Love  absolute music           BLACKPINK
4                       Physical  absolute music            Dua Lipa


In [186]:
song_artist_map = song_artist_map.replace({"genre":{"popular":0, "rock":1, "folk":2, "hiphop":3, "rnb":4, "jazz":5, "electronic":6, "classical": 7, "absolute music":8}})
print(song_artist_map)

                           song_name  genre                 artist
0      Dark River - Festival Version    8.0     Sebastian Ingrosso
1                Derp - Original Mix    8.0            Bassjackers
2           One Kiss (with Dua Lipa)    8.0          Calvin Harris
3                     Kill This Love    8.0              BLACKPINK
4                           Physical    8.0               Dua Lipa
...                              ...    ...                    ...
10092                      Otherside    1.0  Red Hot Chili Peppers
10093          White Wedding - Pt. 1    1.0             Billy Idol
10094            Eyes Without A Face    1.0             Billy Idol
10095                     Rebel Yell    1.0             Billy Idol
10096                            NaN    NaN                    NaN

[10097 rows x 3 columns]


In [187]:
PIK = "artist_pickle.dat"

with open(PIK, "wb") as f:
    pickle.dump(song_artist_map, f)
with open(PIK, "rb") as f:
    reload_song_artist_map = pickle.load(f)

reload_song_artist_map.head(10)

,song_name,genre,artist
0,Dark River - Festival Version,8.0,Sebastian Ingrosso
1,Derp - Original Mix,8.0,Bassjackers
2,One Kiss (with Dua Lipa),8.0,Calvin Harris
3,Kill This Love,8.0,BLACKPINK
4,Physical,8.0,Dua Lipa
5,Turn Me On (feat. Vula),8.0,Riton
6,Lightning Strikes,8.0,Steve Aoki
7,Invincible - Original Mix,8.0,Borgeous
8,Bella ciao - HUGEL Remix,8.0,El Profesor
9,SHIVA (Sunburn 2017 Anthem) [feat. The Golden ...,8.0,KSHMR


## CREATING CORE

In [188]:
# core = spoti[["genre","mode","duration_ms"]]
core = spoti[["genre","mode","danceability", "energy"]]
print(core.dtypes)
print(core.head())

genre            object
mode            float64
danceability    float64
energy          float64
dtype: object
            genre  mode  danceability  energy
0  absolute music   0.0         0.741   0.739
1  absolute music   1.0         0.649   0.973
2  absolute music   0.0         0.791   0.862
3  absolute music   1.0         0.738   0.861
4  absolute music   1.0         0.647   0.844


In [189]:
core["genre"].value_counts()

popular           1867
rock              1340
folk              1271
hiphop            1141
rnb               1086
jazz              1065
electronic        1052
classical          740
absolute music     534
Name: genre, dtype: int64

In [190]:
core = core.replace({"genre":{"popular":0, "rock":1, "folk":2, "hiphop":3, "rnb":4, "jazz":5, "electronic":6, "classical": 7, "absolute music":8}})

In [191]:
from sklearn.impute import SimpleImputer
def replace_missing_value(df, number_features):

    imputer = SimpleImputer(missing_values=np.nan,strategy="median")
    df_num = df[number_features]
    imputer.fit(df_num)
    X = imputer.transform(df_num)
    res_def = pd.DataFrame(X, columns=df_num.columns)
    return res_def

number_features = ["genre","mode","danceability", "energy"]
core = replace_missing_value(core, number_features)

In [192]:
print(core)

       genre  mode  danceability  energy
0        8.0   0.0         0.741   0.739
1        8.0   1.0         0.649   0.973
2        8.0   0.0         0.791   0.862
3        8.0   1.0         0.738   0.861
4        8.0   1.0         0.647   0.844
...      ...   ...           ...     ...
10092    1.0   1.0         0.450   0.525
10093    1.0   1.0         0.338   0.340
10094    1.0   1.0         0.469   0.561
10095    1.0   0.0         0.458   0.795
10096    3.0   1.0         0.592   0.577

[10097 rows x 4 columns]


In [193]:
import pickle
PIK = "core_pickle.dat"

with open(PIK, "wb") as f:
    pickle.dump(core, f)
with open(PIK, "rb") as f:
    reload_core = pickle.load(f)

print(reload_core)

       genre  mode  danceability  energy
0        8.0   0.0         0.741   0.739
1        8.0   1.0         0.649   0.973
2        8.0   0.0         0.791   0.862
3        8.0   1.0         0.738   0.861
4        8.0   1.0         0.647   0.844
...      ...   ...           ...     ...
10092    1.0   1.0         0.450   0.525
10093    1.0   1.0         0.338   0.340
10094    1.0   1.0         0.469   0.561
10095    1.0   0.0         0.458   0.795
10096    3.0   1.0         0.592   0.577

[10097 rows x 4 columns]


# Train the model

### APPLYING NMF

Non-negative matrix factorization (NMF), also non-negative matrix approximation is a group of algorithms in multivariate analysis and linear algebra where a matrix V is factorized into (usually) two matrices W and H, with the property that all three matrices have no negative elements. This non-negativity makes the resulting matrices easier to inspect.

In [194]:
from sklearn.decomposition import NMF

nmf = NMF(n_components = 6)
nmf_features = nmf.fit_transform(core)

In [195]:
from sklearn.preprocessing import normalize
norm_features = normalize(nmf_features)


In [196]:
import pickle
PIK = "nmf_pickle.dat"

with open(PIK, "wb") as f:
    pickle.dump(norm_features, f)
with open(PIK, "rb") as f:
    reload_nmf_features = pickle.load(f)

print(reload_nmf_features)

[[9.94196379e-01 1.05500265e-06 4.58090214e-02 9.72457159e-02
  4.28525372e-03 0.00000000e+00]
 [9.89234187e-01 5.00959954e-02 2.30354971e-02 1.31275610e-01
  0.00000000e+00 3.37963612e-02]
 [9.91703850e-01 4.92369898e-07 5.05425963e-02 1.18145285e-01
  3.25745836e-03 0.00000000e+00]
 ...
 [5.02522072e-01 3.73028201e-02 1.71218289e-01 1.80776362e-01
  3.46640851e-01 7.50948994e-01]
 [1.23263501e-01 0.00000000e+00 2.91623579e-01 9.43498629e-01
  9.78372887e-02 2.82915045e-06]
 [9.26242911e-01 1.46436758e-02 9.71551926e-02 1.37229354e-01
  8.82830301e-02 3.25260989e-01]]


In [197]:
current_music = norm_features[23,:]
similarities = norm_features.dot(current_music)
similarities

array([0.99974129, 0.99794602, 0.9999398 , ..., 0.52689092, 0.24601402,
       0.9391303 ])

In [198]:
current_music = reload_nmf_features[23,:]
similarities = reload_nmf_features.dot(current_music)
similarities

array([0.99974129, 0.99794602, 0.9999398 , ..., 0.52689092, 0.24601402,
       0.9391303 ])

In [199]:
df = pd.DataFrame(norm_features)
x = df.join(song_name)
df = pd.pivot_table(x, x[[0,1,2,3,4,5]],["Song-Names"]) # for indexing song_name to our df

def current_music(value):
    print("Top 5 recommendations for given music are:")
    value = df.loc[value]
    similarities = df.dot(value)
    print(format(similarities.nlargest()))

In [200]:
current_music("The Smile Song")

Top 5 recommendations for given music are:
Song-Names
The Smile Song             1.000000
The Power Of Love          0.999588
Sometimes When We Touch    0.998662
¿ÉÊÇ»¹Ï²»¶                 0.997611
Rainbow Rocks              0.996911
dtype: float64


In [201]:
df = pd.DataFrame(norm_features)
x = df.join(song_name)
df = pd.pivot_table(x, x[[0,1,2,3,4,5]],["Song-Names"]) # for indexing song_name to our df

def current_music(value):
    value = df.loc[value]
    similarities = df.dot(value)
    mylist = similarities.nlargest(10)
    # print(format(similarities.nlargest(10)))

    test = mylist.index.values.tolist()
    return test

In [202]:
list = current_music("Venom")
print(type(list))
print(list)

<class 'list'>
['Stuck In The Middle With You', 'Smoke on the Water', 'One Small Thing', 'Personal Jesus - Original Single Version', 'Paris of the East', "You'll Play Your Part", '3 Libras', 'Hungry Eyes - From "Dirty Dancing" Soundtrack', 'Dalam Diam Aku Mencintaimu', '¤Ï¤¸¤á¤Þ¤·¤Æ¤Î\x9aÝ³Ö¤Á¤ò']
